In [ ]:
# Imports

import time
import socket
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
from matplotlib import rcParams
import pandas as pd
import zlib
import os
import folium

sync = os.system('w32tm /resync')

In [ ]:
# Global parameters

PACKETSIZE = 125

IP = '192.168.1.100'

PORT = 10000

READ_PACKETS = 20

TICKS_1000MS = 60000

# 0 for x, 1 for y, 2 for z

PLOT_AXIS = 0 

In [ ]:
# Enable TCP listener

#IP = socket.gethostbyname(socket.gethostname())

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

sock.bind((IP,PORT))

sock.listen()

print('Listening on %s:%d' % (IP,PORT))

conn, addr = sock.accept()

print('Receiving from %s:%d' % (addr[0],addr[1]))

conn.settimeout(5)

In [ ]:
# Read initial response

running = True

while running:
    try:
        data = conn.recv(100)
        print(data.decode()[:-1])
    except socket.timeout:
        running = False

In [ ]:
# Start streaming

sent = conn.send(bytes('start\n','utf-8'))

In [ ]:
# Read packets

all_t = np.empty(0)
all_a = np.empty((0,3))

stats_packetbytes = np.empty(0).astype(int)
stats_packetsize = np.empty(0).astype(int)
stats_start = np.empty(0)
stats_dur = np.empty(0)
stats_gap = np.empty(0)
stats_gap = np.append(stats_gap,0)
stats_sysdatetime = np.empty(0)
stats_crc32 = np.empty(0).astype(int)
stats_crc32_calc = np.empty(0).astype(int)

# extended packet

stats_lat = np.empty(0).astype(float)
stats_lng = np.empty(0).astype(float)
stats_alt = np.empty(0).astype(int)
stats_vel = np.empty(0).astype(int)
stats_rssi = np.empty(0).astype(int)

stats_ID = np.empty(0).astype(str)
stats_num = np.empty(0).astype(int)

running = True

n = 0

while running:
    
    try:
        
        packet = conn.recv(10 + 9 * PACKETSIZE + 12 + 4 + 8)
        
        stats_sysdatetime = np.append(stats_sysdatetime,datetime.now())
        
        n = n+1
        
        packetsize = (len(packet)-10-12-4-8)//9
        
        start = datetime.fromtimestamp( int.from_bytes(packet[0:4], 'big') ) + timedelta( seconds = int.from_bytes(packet[4:6], 'big') / TICKS_1000MS )

        dur = timedelta( seconds = int.from_bytes(packet[6:10], 'big') / TICKS_1000MS )
        lat = int.from_bytes(packet[-24:-20], 'big');
        lng = int.from_bytes(packet[-20:-16], 'big');
        alt = int.from_bytes(packet[-16:-14], 'big');
        vel = int(packet[-14]);
        rssi = int(packet[-13]);
        
        # received checksum
        
        crc32 = int.from_bytes(packet[-12:-8], 'big')
        
        # calculated checksum
        
        crc32_calc = zlib.crc32(packet[:-12]) & 0xffffffff
        
        # ID
        
        ID = packet[-8:-2].decode('utf-8')
        
        # packet number
        
        num = int.from_bytes(packet[-2:], 'big');
        
        # print message
        
        s = 'Packet : %s %03d, size : %d, time : %s, CRC32 read : %08X, CRC32 calc : %08X' % (ID,num,len(packet),start.strftime('%d/%m/%Y %H:%M:%S.%f'),crc32,crc32_calc)
        
        print(s, end = '\r', flush = True)
        
        # time values (assume equal timestep)
        
        step = dur / (packetsize)
        
        t = np.array([start + i * step for i in range(1,packetsize+1)]) 
        
        # acceleration values
        
        a = np.zeros((packetsize,3))
        
        for col in range(0,3):
            
            line = 0
            
            for j in range(10+3*col, len(packet)-4, 9):

                val = (packet[j] << 12) | (packet[j+1] << 4) | (packet[j+2] >> 4)

                if val >= 524288:
                    
                    val = val - 1048576
                
                try:
                    
                    a[line][col] = val / 256000
                
                    line = line+1 
                    
                except:
                    
                    ;
                    
        # append to time and acceleration arrays
        
        all_t = np.append(all_t,t)
        all_a = np.vstack((all_a,1000*a))
        
        # append to total arrays
        
        stats_start = np.append(stats_start,start)
        stats_packetbytes = np.append(stats_packetbytes,(len(packet)))
        stats_packetsize = np.append(stats_packetsize,packetsize)
        stats_dur = np.append(stats_dur,dur)
        
        gap = (all_t[len(all_t)-packetsize] - all_t[len(all_t)-packetsize-1]).total_seconds()
        
        if len(all_t) > packetsize:
            stats_gap = np.append(stats_gap, gap)
            
        stats_lat = np.append(stats_lat, lat/1e6)
        stats_lng = np.append(stats_lng, lng/1e6)
        stats_alt = np.append(stats_alt, alt)
        stats_vel = np.append(stats_vel, vel)
        stats_rssi = np.append(stats_rssi, -rssi)
        
        stats_crc32 = np.append(stats_crc32, crc32)
        stats_crc32_calc = np.append(stats_crc32_calc, crc32_calc)
        
        stats_ID = np.append(stats_ID, ID)
        stats_num = np.append(stats_num, num)
        
        # stop when finished
        
        if n == READ_PACKETS:
            
            running = False
  
    except socket.timeout:
        
        running = False

In [ ]:
# Stop streaming

sent = conn.send(bytes('stop\n','utf-8'))

time.sleep(1)

conn.close()

sock.close()

In [ ]:
#
# Plot accelerogram
#

plt.figure(figsize = (12, 4))
rcParams['font.family'] = 'isocpeur'

plt.grid()
plt.title('SeismoBug P listener', fontsize = 13)
plt.xlabel('time', fontsize = 11)
plt.ylabel('acceleration (mg)', fontsize = 11)

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
plt.xticks(fontsize = 9, rotation = 45)
plt.yticks(fontsize = 9)

amax = 1.1*max(abs(np.max(all_a[:,PLOT_AXIS])),abs(np.min(all_a[:,PLOT_AXIS])))

#plt.ylim([-amax,amax])

# plot packets with alternating colors

color_toggle = True

for i in range(0, len(all_t), packetsize):
    
    t_seg = all_t[i:i+packetsize]
    
    a_seg = all_a[i:i+packetsize,PLOT_AXIS]
    
    if color_toggle:
        plt.plot(t_seg, a_seg, color = 'royalblue', linewidth = 0.2)
    else:
        plt.plot(t_seg, a_seg, color = 'darkorange', linewidth = 0.2)

    color_toggle = not color_toggle

plt.show()

In [ ]:
#
# Save record
#

all_date = np.array([t.strftime('%d/%m/%Y') for t in all_t])
all_time = np.array([t.strftime('%H:%M:%S.%f') for t in all_t])

df = pd.DataFrame(all_date, columns = ['Date'])

df['Time'] = all_time
df['Acc x'] = all_a[:,0]
df['Acc y'] = all_a[:,1]
df['Acc z'] = all_a[:,2]

df['Acc x'] = df['Acc x'].apply(lambda x: '{:.6f}'.format(x))
df['Acc y'] = df['Acc y'].apply(lambda x: '{:.6f}'.format(x))
df['Acc z'] = df['Acc z'].apply(lambda x: '{:.6f}'.format(x))

df.to_csv('Record.txt', sep = '\t', index = False)

#
# Save to SeismoBug Browser compatible format
#

df_SB = pd.DataFrame();

df_SB['Time (s)'] = np.array([(t-all_t[0]).total_seconds() for t in all_t])               
df_SB['x (g)'] = all_a[:,0] / 1000
df_SB['y (g)'] = all_a[:,1] / 1000
df_SB['z (g)'] = all_a[:,2] / 1000
                     
df_SB.to_csv('Record_SeismoBugBrowser.txt', sep = '\t', index = False)
            
#
# Save packet stats
#

stats_start_date = np.array([t.strftime('%d/%m/%Y') for t in stats_start])
stats_start_time = np.array([t.strftime('%H:%M:%S.%f') for t in stats_start])
stats_systime = np.array([t.strftime('%H:%M:%S.%f') for t in stats_sysdatetime])
stats_dur_secs = np.array([t.total_seconds() for t in stats_dur])

# number
df = pd.DataFrame(stats_num, columns = ['#'])
# ID
df['ID'] = stats_ID
# Packet size
df['Size'] = stats_packetsize
# Packet length
df['Bytes'] = stats_packetbytes
# Packet date
df['Date'] = stats_start_date
# Packet time
df['Time'] = stats_start_time
# Packet duration (as received)
df['Dur read'] = 1000*stats_dur_secs
# Packet calculated duration (current packet time - previous packet time)
df['Dur calc'] = np.concatenate(( 1000*np.array([td.total_seconds() for td in np.diff(stats_start)]) , np.array([0]) ))
# Calculated samples per second from packet duration and packet size
df['SPS'] = stats_packetsize/stats_dur_secs
# Calculated sample duration from packet duration and packet size
df['dt'] = 1000*stats_dur_secs/stats_packetsize
# Time gap between current and previous packet
df['Gap'] = 1000*stats_gap
# System time when packet is received
df['PC time'] = stats_systime
# Time difference between system time and packet time
df['Diff'] = 1000*np.array([t.total_seconds() for t in stats_sysdatetime - stats_start])
# lat
df['Lat'] = stats_lat;
# lng
df['Lng'] = stats_lng;
# altitude
df['Alt'] = stats_alt;
# velocitu
df['Vel'] = stats_vel;
# rssi
df['RSSI'] = stats_rssi;
# Packet CRC32
df['CRC32 recv'] = stats_crc32
# Packet CRC32 calculated
df['CRC32 calc'] = stats_crc32_calc
# check
df['Check'] = np.where(df['CRC32 recv'] == df['CRC32 calc'], 'Ok', 'Corrupt')


# Data formatting

df['Dur read'] = df['Dur read'].apply(lambda x: '{:.2f}'.format(x))
df['Dur calc'] = df['Dur calc'].apply(lambda x: '{:.2f}'.format(x))
df['SPS'] = df['SPS'].apply(lambda x: '{:.2f}'.format(x))
df['dt'] = df['dt'].apply(lambda x: '{:.2f}'.format(x))
df['Gap'] = df['Gap'].apply(lambda x: '{:.2f}'.format(x))
df['Diff'] = df['Diff'].apply(lambda x: '{:.2f}'.format(x))
df['Lat'] = df['Lat'].apply(lambda x: '{:.6f}'.format(x))
df['Lng'] = df['Lng'].apply(lambda x: '{:.6f}'.format(x))
df['CRC32 recv'] = df['CRC32 recv'].apply(lambda x: '{:08X}'.format(x))
df['CRC32 calc'] = df['CRC32 calc'].apply(lambda x: '{:08X}'.format(x))

# Save

df.to_csv('Packets.txt', sep = '\t', index = False)

In [ ]:
#
# Show record stats
#

print('Mean SPS = %.2f' % np.mean(stats_packetsize/stats_dur_secs))
print('StDev SPS = %.4f' % np.std(stats_packetsize/stats_dur_secs))
print()      
print('Mean x = %.3f mg' % np.mean(all_a[:,0]))
print('Mean y = %.3f mg' % np.mean(all_a[:,1]))
print('Mean z = %.3f mg' % np.mean(all_a[:,2]))
print()      
print('StDev x = %.3f mg' % np.std(all_a[:,0]))
print('StDev y = %.3f mg' % np.std(all_a[:,1]))
print('StDev z = %.3f mg' % np.std(all_a[:,2]))
print()
print('RMS x = %.3f mg' % np.sqrt(np.mean(all_a[:,0]**2)))
print('RMS y = %.3f mg' % np.sqrt(np.mean(all_a[:,1]**2)))
print('RMS z = %.3f mg' % np.sqrt(np.mean(all_a[:,2]**2)))
print()
print('p-p x = %.3f mg' % (np.max(all_a[:,0]) - np.min(all_a[:,0])) )
print('p-p y = %.3f mg' % (np.max(all_a[:,1]) - np.min(all_a[:,1])) )
print('p-p z = %.3f mg' % (np.max(all_a[:,2]) - np.min(all_a[:,2])) )

In [ ]:
def get_color(rssi):
    
    cmap = plt.get_cmap('jet')
    norm = plt.Normalize(30,100)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    rgba_color = sm.to_rgba(-rssi)
    return "#{:02x}{:02x}{:02x}".format(int(rgba_color[0]*255), int(rgba_color[1]*255), int(rgba_color[2]*255))

m = folium.Map(location=[np.mean(stats_lat), np.mean(stats_lng)], zoom_start=18)

for lat, lon, r in zip(stats_lat, stats_lng, stats_rssi):
    
    c = get_color(r)
    
    folium.CircleMarker(location=(lat, lon), radius=10, color="black", weight=0.3, fill=True, fill_color=c, fill_opacity=0.5, popup=f'RSSI: {r}').add_to(m)
    
m